In [2]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
class h_sigmoid(nn.Module):
    def __init__(self, inplace=True):
        super(h_sigmoid, self).__init__()
        self.relu = nn.ReLU6(inplace=inplace)

    def forward(self, x):
        return self.relu(x + 3) / 6

class h_swish(nn.Module):
    def __init__(self, inplace=True):
        super(h_swish, self).__init__()
        self.sigmoid = h_sigmoid(inplace=inplace)

    def forward(self, x):
        return x * self.sigmoid(x)

class CoordAtt(nn.Module):
    def __init__(self, inp, oup, reduction=32):
        super(CoordAtt, self).__init__()
        self.pool_h = nn.AdaptiveAvgPool2d((None, 1))
        self.pool_w = nn.AdaptiveAvgPool2d((1, None))

        mip = max(8, inp // reduction)

        self.conv1 = nn.Conv2d(inp, mip, kernel_size=1, stride=1, padding=0)
        self.bn1 = nn.BatchNorm2d(mip)
        self.act = h_swish()
        
        self.conv_h = nn.Conv2d(mip, oup, kernel_size=1, stride=1, padding=0)
        self.conv_w = nn.Conv2d(mip, oup, kernel_size=1, stride=1, padding=0)
        

    def forward(self, x):
        identity = x
        
        n,c,h,w = x.size()
        x_h = self.pool_h(x)
        x_w = self.pool_w(x).permute(0, 1, 3, 2)

        y = torch.cat([x_h, x_w], dim=2)
        y = self.conv1(y)
        y = self.bn1(y)
        y = self.act(y) 
        
        x_h, x_w = torch.split(y, [h, w], dim=2)
        x_w = x_w.permute(0, 1, 3, 2)

        a_h = self.conv_h(x_h).sigmoid()
        a_w = self.conv_w(x_w).sigmoid()

        out = identity * a_w * a_h
#         print(out.shape)
        return out

In [4]:
model=CoordAtt(256,256)
x1=torch.rand((32,256,16,16))
x=model(x1)

In [5]:
x.shape

torch.Size([32, 256, 16, 16])

In [48]:
class BaseConv(nn.Module):
    """A Conv2d -> Batchnorm -> silu/leaky relu block"""

    def __init__(
        self, in_channels, out_channels, ksize, stride, groups=1, bias=False, act="silu"
    ):
        super().__init__()
        # same padding
        pad = (ksize - 1) // 2
        self.conv = nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size=ksize,
            stride=stride,
            padding=pad,
            groups=groups,
            bias=bias,
        )
        self.bn = nn.BatchNorm2d(out_channels)
        self.act = nn.ReLU(inplace=True)

    def forward(self, x):
        return self.act(self.bn(self.conv(x)))

    def fuseforward(self, x):
        return self.act(self.conv(x))
    
class test(nn.Module):
    def __init__(self):
        super(test, self).__init__()
        base_channels=64
        self.conv1=BaseConv(base_channels*4, base_channels*16, 3, 4)
        self.conv2=BaseConv(base_channels*8, base_channels*16, 3, 2)
        self.ca1=CoordAtt(base_channels*4,base_channels*4)
        self.ca2=CoordAtt(base_channels*8,base_channels*8)
        self.conv=BaseConv(base_channels*16, base_channels*16, 3, 1)
    def forward(self, x1,x2,x3):
        print(x1.shape)
        x1=self.ca1(x1)
        print(x1.shape)
        x1=self.conv1(x1)
        print(x1.shape)
        x2=self.ca2(x2)
        x2=self.conv2(x2)
        
        x=x1+x2+x3
        x=self.conv(x)
        print(x.shape)
        return x

In [49]:
model=test()
x1=torch.rand((32,256,16,16))
x2=torch.rand((32,512,8,8))
x3=torch.rand((32,1024,4,4))
x=model(x1,x2,x3)

torch.Size([32, 256, 16, 16])
torch.Size([32, 256, 16, 16])
torch.Size([32, 1024, 4, 4])
torch.Size([32, 1024, 4, 4])


In [32]:
x.shape

torch.Size([32, 1024, 4, 4])

In [5]:
class BaseConv(nn.Module):
    """A Conv2d -> Batchnorm -> silu/leaky relu block"""

    def __init__(
        self, in_channels, out_channels, ksize, stride, groups=1, bias=False, act="silu"
    ):
        super().__init__()
        # same padding
        pad = (ksize - 1) // 2
        self.conv = nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size=ksize,
            stride=stride,
            padding=pad,
            groups=groups,
            bias=bias,
        )
        self.bn = nn.BatchNorm2d(out_channels)
        self.act = nn.ReLU(inplace=True)

    def forward(self, x):
        return self.act(self.bn(self.conv(x)))

    def fuseforward(self, x):
        return self.act(self.conv(x))
    
class test(nn.Module):
    def __init__(self):
        super(test, self).__init__()
        base_channels=64
        self.upsample = nn.Upsample(scale_factor=2, mode="nearest")
        self.conv3=BaseConv(base_channels*16, base_channels*8, 3, 1)
        self.conv2=BaseConv(base_channels*8, base_channels*4, 3, 1)
        
        self.conv1=BaseConv(base_channels*4, base_channels*8, 3, 2)
        self.conv2_=BaseConv(base_channels*8, base_channels*16, 3, 2)
    def forward(self, x1,x2,x3):
        x3=self.upsample(x3)
        x3=self.conv3(x3)
        x2=x3+x2
        x2=self.upsample(x2)
        x2=self.conv2(x2)
        x1=x2+x1
        
        y2=self.conv1(x1)
        y2=y2+
        y2=self.conv2_(x2)
        return x1

In [6]:
model=test()
x1=torch.rand((32,256,16,16))
x2=torch.rand((32,512,8,8))
x3=torch.rand((32,1024,4,4))
x=model(x1,x2,x3)

In [7]:
x.shape

torch.Size([32, 256, 16, 16])

In [32]:
def get_spp_size(h,w, out_pool_size):
    h_wid = math.ceil(h / out_pool_size)
    w_wid = math.ceil(w / out_pool_size)
    h_str = math.floor(h / out_pool_size)
    w_str = math.floor(w / out_pool_size)
    return h_wid,w_wid,h_str,w_str
class BaseConv(nn.Module):
    """A Conv2d -> Batchnorm -> silu/leaky relu block"""

    def __init__(
        self, in_channels, out_channels, ksize, stride, groups=1, bias=False, act="silu"
    ):
        super().__init__()
        # same padding
        pad = (ksize - 1) // 2
        self.conv = nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size=ksize,
            stride=stride,
            padding=pad,
            groups=groups,
            bias=bias,
        )
        self.bn = nn.BatchNorm2d(out_channels)
        self.act = nn.ReLU(inplace=True)

    def forward(self, x):
        return self.act(self.bn(self.conv(x)))

    def fuseforward(self, x):
        return self.act(self.conv(x))
    
class test(nn.Module):
    def __init__(self):
        super(test, self).__init__()
        base_channels=64
        self.conv1=BaseConv(base_channels*4, base_channels*16, ksize=1, stride=1)
        self.pool1=nn.AdaptiveAvgPool2d(1)
        
        self.conv2=BaseConv(base_channels*8, base_channels*16, ksize=1, stride=1)
        self.pool2=nn.AdaptiveAvgPool2d(1)
        
        self.pool3=nn.AdaptiveAvgPool2d(1)
        self.bn = nn.BatchNorm1d(1)
        self.act = nn.ReLU(inplace=True)
        b=1
        gamma=2
        channel=base_channels*16
        kernel_size = int(abs((math.log(channel, 2) + b) / gamma))
        kernel_size = kernel_size if kernel_size % 2 else kernel_size + 1
        print(kernel_size)
        self.conv = nn.Conv1d(1, 1, kernel_size=kernel_size, padding=(kernel_size - 1) // 2, bias=False)
    def forward(self, x1,x2,x3):
        b,_,_,_=x1.size()
        x1=self.conv1(x1)
        x1=self.pool1(x1)
        
        print(x1.shape)
        
        x2=self.conv2(x2)
        x2=self.pool2(x2)
        print(x2.shape)
        
        x3=self.pool3(x3)
        print(x3.shape)
        x=x1+x2+x3
        
#         x=torch.cat((x1,x2,x3), 1).unsqueeze(1)
#         x=self.conv(x).transpose(-1, -2).unsqueeze(-1)
#         x=self.conv(x)
        print(x.shape)
        x=x.squeeze(-1).transpose(-1, -2)
        x=self.bn(x)
        x=self.act(x).transpose(-1, -2).unsqueeze(-1)
        print(x.shape)
        return x1,x2,x3,x

In [33]:
model=test()
x1=torch.rand((32,256,16,16))
x2=torch.rand((32,512,8,8))
x3=torch.rand((32,1024,4,4))
x1,x2,x3,x=model(x1,x2,x3)

5
torch.Size([32, 1024, 1, 1])
torch.Size([32, 1024, 1, 1])
torch.Size([32, 1024, 1, 1])
torch.Size([32, 1024, 1, 1])
torch.Size([32, 1024, 1, 1])


In [11]:
def get_spp_size(h,w, out_pool_size):
    h_wid = math.ceil(h / out_pool_size)
    w_wid = math.ceil(w / out_pool_size)
    h_str = math.floor(h / out_pool_size)
    w_str = math.floor(w / out_pool_size)
    return h_wid,w_wid,h_str,w_str
class BaseConv(nn.Module):
    """A Conv2d -> Batchnorm -> silu/leaky relu block"""

    def __init__(
        self, in_channels, out_channels, ksize, stride, groups=1, bias=False, act="silu"
    ):
        super().__init__()
        # same padding
        pad = (ksize - 1) // 2
        self.conv = nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size=ksize,
            stride=stride,
            padding=pad,
            groups=groups,
            bias=bias,
        )
        self.bn = nn.BatchNorm2d(out_channels)
        self.act = nn.ReLU(inplace=True)

    def forward(self, x):
        return self.act(self.bn(self.conv(x)))

    def fuseforward(self, x):
        return self.act(self.conv(x))
    
class test(nn.Module):
    def __init__(self):
        super(test, self).__init__()
        base_channels=64
        self.conv1=BaseConv(base_channels*4, base_channels*16, ksize=1, stride=1)
        self.pool1=nn.AdaptiveAvgPool2d(1)
        
        self.conv2=BaseConv(base_channels*8, base_channels*16, ksize=1, stride=1)
        self.pool2=nn.AdaptiveAvgPool2d(1)
        
        self.pool3=nn.AdaptiveAvgPool2d(1)
        
        b=1
        gamma=2
        channel=base_channels*16
        kernel_size = int(abs((math.log(channel, 2) + b) / gamma))
        kernel_size = kernel_size if kernel_size % 2 else kernel_size + 1
        print(kernel_size)
        self.conv = nn.Conv1d(1, 1, kernel_size=kernel_size, padding=(kernel_size - 1) // 2, bias=False)
    def forward(self, x1,x2,x3):
        b,_,_,_=x1.size()
        
        x1=self.pool1(x1)
        x1=self.conv1(x1)
        print(x1.shape)
        
        x2=self.conv2(x2)
        x2=self.pool2(x2).view(b, -1)
        
        x3=self.pool3(x3).view(b, -1)
        x=torch.cat((x1,x2,x3), 1).unsqueeze(1)
#         x=self.conv(x).transpose(-1, -2).unsqueeze(-1)
#         x=self.conv(x)
        
        return x1,x2,x3,x

In [12]:
model=test()
x1=torch.rand((32,256,16,16))
x2=torch.rand((32,512,8,8))
x3=torch.rand((32,1024,4,4))
x1,x2,x3,x=model(x1,x2,x3)

5
torch.Size([32, 1024, 1, 1])


RuntimeError: Tensors must have same number of dimensions: got 4 and 2

In [8]:
def get_spp_size(h,w, out_pool_size):
    h_wid = math.ceil(h / out_pool_size)
    w_wid = math.ceil(w / out_pool_size)
    h_str = math.floor(h / out_pool_size)
    w_str = math.floor(w / out_pool_size)
    return h_wid,w_wid,h_str,w_str
class BaseConv(nn.Module):
    """A Conv2d -> Batchnorm -> silu/leaky relu block"""

    def __init__(
        self, in_channels, out_channels, ksize, stride, groups=1, bias=False, act="silu"
    ):
        super().__init__()
        # same padding
        pad = (ksize - 1) // 2
        self.conv = nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size=ksize,
            stride=stride,
            padding=pad,
            groups=groups,
            bias=bias,
        )
        self.bn = nn.BatchNorm2d(out_channels)
        self.act = nn.ReLU(inplace=True)

    def forward(self, x):
        return self.act(self.bn(self.conv(x)))

    def fuseforward(self, x):
        return self.act(self.conv(x))
    
class test(nn.Module):
    def __init__(self):
        super(test, self).__init__()
        base_channels=64
        self.conv1=BaseConv(base_channels*4, base_channels*16, ksize=1, stride=1)
        size1=get_spp_size(16,16,1)
        self.pool1=nn.MaxPool2d(kernel_size=(size1[0], size1[1]), stride=(size1[2], size1[3]))
        
        self.conv2=BaseConv(base_channels*8, base_channels*16, ksize=1, stride=1)
        size1=get_spp_size(8,8,2)
        self.pool2=nn.MaxPool2d(kernel_size=(size1[0], size1[1]), stride=(size1[2], size1[3]))
        
        size1=get_spp_size(4,4,4)
        self.pool3=nn.MaxPool2d(kernel_size=(size1[0], size1[1]), stride=(size1[2], size1[3]))
        b=1
        gamma=2
        channel=base_channels*16
        kernel_size = int(abs((math.log(channel, 2) + b) / gamma))
        kernel_size = kernel_size if kernel_size % 2 else kernel_size + 1
        print(kernel_size)
#         self.conv = nn.Conv1d(1, 1, kernel_size=kernel_size, padding=(kernel_size - 1) // 2, bias=False)
        self.conv=nn.MaxPool1d(kernel_size=4)
    def forward(self, x1,x2,x3):
        b,_,_,_=x1.size()
        x1=self.conv1(x1)
        x1=self.pool1(x1).view(b, -1)
        
        x2=self.conv2(x2)
        x2=self.pool2(x2).view(b, -1)
        
        x3=self.pool3(x3).view(b, -1)
        x=torch.cat((x1,x2,x3), 1).unsqueeze(1)
#         x=self.conv(x).transpose(-1, -2).unsqueeze(-1)
#         x=self.conv(x)
        
        return x1,x2,x3,x

In [9]:
model=test()
x1=torch.rand((32,256,16,16))
x2=torch.rand((32,512,8,8))
x3=torch.rand((32,1024,4,4))
x1,x2,x3,x=model(x1,x2,x3)

5


In [10]:
x1.shape,x2.shape,x3.shape,x.shape

(torch.Size([32, 1024]),
 torch.Size([32, 4096]),
 torch.Size([32, 16384]),
 torch.Size([32, 1, 21504]))

In [5]:
class eca_block(nn.Module):
    def __init__(self, channel, b=1, gamma=2):
        super(eca_block, self).__init__()
        kernel_size = int(abs((math.log(channel, 2) + b) / gamma))
        print(kernel_size)
        kernel_size = kernel_size if kernel_size % 2 else kernel_size + 1
        print(kernel_size)
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.conv = nn.Conv1d(1, 1, kernel_size=kernel_size, padding=(kernel_size - 1) // 2, bias=False) 
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        print(x.shape)
        y = self.avg_pool(x)
        print(y.shape)
        z=y.squeeze(-1).transpose(-1, -2)
        print(z.shape)
        z=self.conv(y.squeeze(-1).transpose(-1, -2))
        print(z.shape)
        y = self.conv(y.squeeze(-1).transpose(-1, -2)).transpose(-1, -2).unsqueeze(-1)
        print(y.shape)
        y = self.sigmoid(y)
        return x * y.expand_as(x)

In [6]:
model=eca_block(256)
input=torch.rand((1,256,64,64))
out=model(input)

4
5
torch.Size([1, 256, 64, 64])
torch.Size([1, 256, 1, 1])
torch.Size([1, 1, 256])
torch.Size([1, 1, 256])
torch.Size([1, 256, 1, 1])


In [ ]:
conv = nn.Conv1d(1, 1, kernel_size=kernel_size, padding=(kernel_size - 1) // 2, bias=False) 

In [37]:
input=torch.rand((32, 86016))

In [40]:
input.view(32,86016,1,1).shape

torch.Size([32, 86016, 1, 1])

In [39]:
input.shape

torch.Size([32, 86016])

In [33]:
input0=torch.rand((32,256,16,16))
input1=torch.rand((32,256,8,8))
input2=torch.rand((32,256,4,4))

In [41]:
b=32
input0=input0.view(b, -1)
input1=input1.view(b, -1)
input2=input2.view(b, -1)
x=torch.cat((input0, input1,input2), 1)

In [45]:
input2.shape

torch.Size([32, 4096])

In [36]:
x.shape

torch.Size([32, 86016])

In [1]:
def get_spp_size(h,w, out_pool_size):
    h_wid = math.ceil(h / out_pool_size)
    w_wid = math.ceil(w / out_pool_size)
    h_str = math.floor(h / out_pool_size)
    w_str = math.floor(w / out_pool_size)
    return h_wid,w_wid,h_str,w_str
    

In [5]:
t=get_spp_size(8,8,2)

In [6]:
t[0]

4

In [28]:
def spatial_pyramid_pool(previous_conv, num_sample, previous_conv_size, out_pool_size):
    """
    previous_conv: a tensor vector of previous convolution layer
    num_sample: an int number of image in the batch
    previous_conv_size: an int vector [height, width] of the matrix features size of previous convolution layer
    out_pool_size: a int vector of expected output size of max pooling layer
    returns: a tensor vector with shape [1 x n] is the concentration of multi-level pooling
"""
    for i in range(len(out_pool_size)):
        # out_pool_size是一个数组，例如[1,2,4],表示需要将原先的特征图分别划分为1×1、2×2、4×4三种.
        h, w = previous_conv_size
        # h,w表示原先特征图的长和宽
        h_wid = math.ceil(h / out_pool_size[i])
        w_wid = math.ceil(w / out_pool_size[i])
        # 计算每一个子块的长和宽，这里需要进行取整
        h_str = math.floor(h / out_pool_size[i])
        w_str = math.floor(w / out_pool_size[i])
        # 计算池化的步长
        max_pool = nn.MaxPool2d(kernel_size=(h_wid, w_wid), stride=(h_str, w_str))
        x = max_pool(previous_conv)
        print(x.shape)
        print(x.view(num_sample, -1).shape)
        # 对每个子块进行最大池化
        if i == 0:
            spp = x.view(num_sample, -1)
#             print(spp.shape)
        else:
            spp = torch.cat((spp, x.view(num_sample, -1)), 1)
#             print(spp.shape)
     # 拼接各个子块的输出结果
    return spp

In [30]:
input=torch.rand((32,256,64,64))
spp=spatial_pyramid_pool(input, 32, [64,64], [1,2,4])


torch.Size([32, 256, 1, 1])
torch.Size([32, 256])
torch.Size([32, 256, 2, 2])
torch.Size([32, 1024])
torch.Size([32, 256, 4, 4])
torch.Size([32, 4096])


In [31]:
spp.shape

torch.Size([32, 5376])

In [7]:
class BaseConv(nn.Module):
    """A Conv2d -> Batchnorm -> silu/leaky relu block"""

    def __init__(
        self, in_channels, out_channels, ksize, stride, groups=1, bias=False, act="silu"
    ):
        super().__init__()
        # same padding
        pad = (ksize - 1) // 2
        self.conv = nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size=ksize,
            stride=stride,
            padding=pad,
            groups=groups,
            bias=bias,
        )
        self.bn = nn.BatchNorm2d(out_channels)
        self.act = get_activation(act, inplace=True)

    def forward(self, x):
        return self.act(self.bn(self.conv(x)))

    def fuseforward(self, x):
        return self.act(self.conv(x))
def get_activation(name="silu", inplace=True):
    if name == "silu":
        module = nn.SiLU(inplace=inplace)
    elif name == "relu":
        module = nn.ReLU(inplace=inplace)
    elif name == "lrelu":
        module = nn.LeakyReLU(0.1, inplace=inplace)
    else:
        raise AttributeError("Unsupported act type: {}".format(name))
    return module



In [13]:
class SPPBottleneck(nn.Module):
    """Spatial pyramid pooling layer used in YOLOv3-SPP"""

    def __init__(
        self, in_channels, out_channels, kernel_sizes=(5, 9, 13), activation="silu"
    ):
        super().__init__()
        hidden_channels = in_channels // 2
        self.conv1 = BaseConv(in_channels, hidden_channels, 1, stride=1, act=activation)
        self.m = nn.ModuleList(
            [
                nn.MaxPool2d(kernel_size=ks, stride=1, padding=ks // 2)
                for ks in kernel_sizes
            ]
        )
        conv2_channels = hidden_channels * (len(kernel_sizes) + 1)
        self.conv2 = BaseConv(conv2_channels, out_channels, 1, stride=1, act=activation)

    def forward(self, x):
        x = self.conv1(x)
        print(x.shape)
        for m in self.m:
            t=m(x)
            print(t.shape)
        x = torch.cat([x] + [m(x) for m in self.m], dim=1)
        x = self.conv2(x)
        return x

In [14]:
model=SPPBottleneck(in_channels=256, out_channels=64)
input=torch.rand((1,256,64,64))
out=model(input)

torch.Size([1, 128, 64, 64])
torch.Size([1, 128, 64, 64])
torch.Size([1, 128, 64, 64])
torch.Size([1, 128, 64, 64])


In [10]:
out.shape

torch.Size([1, 64, 64, 64])

In [5]:
class GAM_Attention(nn.Module):  
    def __init__(self, in_channels, out_channels, rate=4):  
        super(GAM_Attention, self).__init__()  

        self.channel_attention = nn.Sequential(  
            nn.Linear(in_channels, int(in_channels / rate)),  
            nn.ReLU(inplace=True),  
            nn.Linear(int(in_channels / rate), in_channels)  
        )  
      
    def forward(self, x):  
        b, c, h, w = x.shape  
        x_permute = x.permute(0, 2, 3, 1).view(b, -1, c)  
        x_att_permute = self.channel_attention(x_permute).view(b, h, w, c) 
        print(x_att_permute.shape)
        x_channel_att = x_att_permute.permute(0, 3, 1, 2)  
      
        x = x * x_channel_att  
        return x_channel_att,x

In [6]:
if __name__ == '__main__':  
    x = torch.randn(1, 64, 32, 48)  
    b, c, h, w = x.shape  
    net = GAM_Attention(in_channels=c, out_channels=c)  
    y,y2 = net(x)  

torch.Size([1, 32, 48, 64])


In [4]:
 y.shape,y2.shape

(torch.Size([1, 64, 32, 48]), torch.Size([1, 64, 32, 48]))

In [ ]:
import torch.nn as nn  
import torch  


class GAM_Attention(nn.Module):  
    def __init__(self, in_channels, out_channels, rate=4):  
        super(GAM_Attention, self).__init__()  

        self.channel_attention = nn.Sequential(  
            nn.Linear(in_channels, int(in_channels / rate)),  
            nn.ReLU(inplace=True),  
            nn.Linear(int(in_channels / rate), in_channels)  
        )  
      
        self.spatial_attention = nn.Sequential(  
            nn.Conv2d(in_channels, int(in_channels / rate), kernel_size=7, padding=3),  
            nn.BatchNorm2d(int(in_channels / rate)),  
            nn.ReLU(inplace=True),  
            nn.Conv2d(int(in_channels / rate), out_channels, kernel_size=7, padding=3),  
            nn.BatchNorm2d(out_channels)  
        )  
      
    def forward(self, x):  
        b, c, h, w = x.shape  
        x_permute = x.permute(0, 2, 3, 1).view(b, -1, c)  
        x_att_permute = self.channel_attention(x_permute).view(b, h, w, c)  
        x_channel_att = x_att_permute.permute(0, 3, 1, 2)  
      
        x = x * x_channel_att  
      
        x_spatial_att = self.spatial_attention(x).sigmoid()  
        out = x * x_spatial_att  
      
        return out  

if __name__ == '__main__':  
    x = torch.randn(1, 64, 32, 48)  
    b, c, h, w = x.shape  
    net = GAM_Attention(in_channels=c, out_channels=c)  
    y = net(x)  

In [ ]:
self.conv_r = ConvBlock(in_channels=256, out_channels=64, kernel_size=1, stride=1, padding=0)

In [7]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=2, padding=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding,
                               bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()

    def forward(self, input):
        x = self.conv1(input)
        return self.relu(self.bn(x))

In [8]:
model=ConvBlock(in_channels=256, out_channels=64, kernel_size=1, stride=1, padding=0)
input=torch.rand((1,256,64,64))
out=model(input)

In [9]:
out.shape#1, 64, 128, 128

torch.Size([1, 64, 64, 64])

In [ ]:
x_r4 = self.conv_r(x_r4)
x_r3 = self.up_r1(x_r4, x_r3)#3:1,128,128,128->1,64,128,128

In [ ]:
class FCA_(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.bn = nn.BatchNorm2d(out_channels)
        self.sigmoid = nn.Sigmoid()
        self.in_channels = in_channels
        self.avgpool = nn.AdaptiveAvgPool2d(output_size=(1, 1))

    def forward(self, input):
        # global average pooling
        x = self.avgpool(input)
        assert self.in_channels == x.size(1), 'in_channels and out_channels should all be {}'.format(x.size(1))
        x = self.conv(x)
        x = self.sigmoid(x)
        x = torch.mul(input, x)
        return x

In [3]:
class FCA(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.bn = nn.BatchNorm2d(out_channels)
        self.sigmoid = nn.Sigmoid()
        self.in_channels = in_channels
        self.avgpool = nn.AdaptiveAvgPool2d(output_size=(1, 1))

    def forward(self, input):
        # global average pooling
        x = self.avgpool(input)
        assert self.in_channels == x.size(1), 'in_channels and out_channels should all be {}'.format(x.size(1))
        x = self.conv(x)
        x = self.sigmoid(x)
        x = torch.mul(input, x)
        return x

In [5]:
model=FCA(64,64)
input=torch.rand((1,64,64,64))
out=model(input)

In [6]:
out.shape

torch.Size([1, 64, 64, 64])

In [ ]:
class DownsamplingBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size,
                 stride=2, padding=1, dilation=1, groups=1, avg_pool=True,
                 se_block=True, activation=nn.ReLU(),
                 ):

        super().__init__()
        self.groups = groups
        self.stride = stride
        self.kernel_size = kernel_size
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.padding = padding
        self.se_block = se_block

        assert padding == 1
        padding_11 = padding - 3 // 2

        self.fused = False

        self.dense_groups = groups
        self.nonlinearity = activation
        
        self.rbr_dense = conv_bn(
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=3,
            stride=stride,
            padding=padding,
            groups=1) if (kernel_size != 1) else None
        
        self.rbr_1x1 = nn.Sequential(
                nn.AvgPool2d(2, 2),
                conv_bn(in_channels=in_channels, out_channels=out_channels,
                        kernel_size=1, stride=1, padding=0, groups=groups))

        if self.se_block:
            self.se = SE1(
                in_channels, out_channels, g=groups,
                ver=2 if (out_channels != in_channels or stride != 1) else 1)

    def forward(self, inputs):
        rbr_1x1_output=self.rbr_1x1(inputs)
        drop_path_output = self.rbr_dense(inputs)
        out = drop_path_output + rbr_1x1_output 
        out = out * self.se(inputs)############
        out = self.nonlinearity(out)
        return out
    def fuse_conv_bn(self, conv, bn):

        std = (bn.running_var + bn.eps).sqrt()
        bias = bn.bias - bn.running_mean * bn.weight / std

        t = (bn.weight / std).reshape(-1, 1, 1, 1)
        weights = conv.weight * t

        bn = nn.Identity()
        conv = nn.Conv2d(in_channels=conv.in_channels,
                         out_channels=conv.out_channels,
                         kernel_size=conv.kernel_size,
                         stride=conv.stride,
                         padding=conv.padding,
                         dilation=conv.dilation,
                         groups=conv.groups,
                         bias=True,
                         padding_mode=conv.padding_mode)

        conv.weight = torch.nn.Parameter(weights)
        conv.bias = torch.nn.Parameter(bias)

        return conv

    def fuse_repvgg_block(self):
        self.rbr_dense = self.fuse_conv_bn(self.rbr_dense.conv, self.rbr_dense.bn)

        if isinstance(self.rbr_1x1, nn.Sequential) and isinstance(self.rbr_1x1[0], nn.AvgPool2d):
            self.rbr_1x1[1] = self.fuse_conv_bn(self.rbr_1x1[1].conv, self.rbr_1x1[1].bn)
            rbr_1x1_bias = self.rbr_1x1[1].bias

            weight_1x1_expanded = torch.nn.functional.interpolate(self.rbr_1x1[1].weight, scale_factor=2.0, mode='nearest')
            weight_1x1_expanded = weight_1x1_expanded / 4
            weight_1x1_expanded = torch.nn.functional.pad(weight_1x1_expanded, [1, 0, 1, 0])
        else:
            self.rbr_1x1 = self.fuse_conv_bn(self.rbr_1x1.conv, self.rbr_1x1.bn)
            rbr_1x1_bias = self.rbr_1x1.bias

            weight_1x1_expanded = torch.nn.functional.pad(self.rbr_1x1.weight, [1, 1, 1, 1])

        self.rbr_dense.weight = torch.nn.Parameter(self.rbr_dense.weight + weight_1x1_expanded)
        self.rbr_dense.bias = torch.nn.Parameter(self.rbr_dense.bias + rbr_1x1_bias)

        self.rbr_1x1 = nn.Identity()

        self.fused = True


In [2]:
def autopad(k, p=None):  # kernel, padding
    # Pad to 'same'
    if p is None:
        p = k // 2 if isinstance(k, int) else [x // 2 for x in k]  # auto-pad
#         print(p)
    return p
class SE1(nn.Module):
    # Squeeze-and-excitation block in https://arxiv.org/abs/1709.01507
    # ch_in, ch_out, number, shortcut, groups, expansion
    def __init__(self, c_in, c_out, n=1, shortcut=True,  g=1, e=0.5, ver=1):
        super(SE1, self).__init__()
        self.ver = ver
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.cvsig = ConvSig(c_in, c_out, 1, 1, g=g)

    def forward(self, x):
        x = self.cvsig(self.avg_pool(x))
        if self.ver == 2:
            x = 2 * x
        return x
def conv_bn(in_channels, out_channels, kernel_size, stride, padding, groups=1):
    result = nn.Sequential()
    result.add_module(
        'conv',
        nn.Conv2d(
            in_channels=in_channels, out_channels=out_channels,
            kernel_size=kernel_size, stride=stride, padding=padding,
            groups=groups, bias=False))
    result.add_module('bn', nn.BatchNorm2d(num_features=out_channels))
    return result
class ConvSig(nn.Module):
    # Standard convolution
    def __init__(self, c1, c2, k=1, s=1, p=None, g=1, act=True):  # ch_in, ch_out, kernel, stride, padding, groups
        super(ConvSig, self).__init__()
        self.conv = nn.Conv2d(c1, c2, k, s, autopad(k, p), groups=g, bias=False)
        self.act = nn.Sigmoid() if act else nn.Identity()
#         print(autopad(k, p))
    def forward(self, x):
        
        return self.act(self.conv(x))



In [40]:
model=Encoder(imageSize=128, nz=100, nc=3, ngf=64, ngpu=1, n_extra_layers=0, add_final_conv=True)
input=torch.rand((1,3,128,128))
out,y=model(input)

torch.Size([1, 512, 8, 8])


In [41]:
out.shape,y.shape

(torch.Size([1, 1024, 4, 4]), torch.Size([1, 100, 1, 1]))

In [ ]:
class RepVGGBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size,
                 stride=1, padding=0, dilation=1, groups=1,
                 padding_mode='zeros', avg_pool=True,
                 se_block=True, activation=nn.SiLU(),
                 ):

        super().__init__()
        self.groups = groups
        self.stride = stride
        self.kernel_size = kernel_size
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.padding = padding
        self.padding_mode = padding_mode
        self.se_block = se_block

        assert padding == 1
        padding_11 = padding - 3 // 2

        self.fused = False

        self.dense_groups = groups
        self.nonlinearity = activation

        self.rbr_identity = nn.BatchNorm2d(
            num_features=in_channels) if (
                out_channels == in_channels and
                stride == 1) else None
        self.rbr_dense = conv_bn(
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=3,
            stride=stride,
            padding=padding,
            groups=self.dense_groups) if (kernel_size != 1) else None
        self.rbr_1x1 = conv_bn(
            in_channels=in_channels,
            out_channels=out_channels,
            kernel_size=1,
            stride=stride,
            padding=padding_11,
            groups=groups)
        if stride == 2 and avg_pool:
            self.rbr_1x1 = nn.Sequential(
                nn.AvgPool2d(2, 2),
                conv_bn(in_channels=in_channels, out_channels=out_channels,
                        kernel_size=1, stride=1, padding=0, groups=groups)
            )

        # updated to reuse code
        self.channel_shuffle = (groups > 1)

        if self.se_block:
            self.se = SE1(
                in_channels, out_channels, g=groups,
                ver=2 if (out_channels != in_channels or stride != 1) else 1)

    def _forward(self, inputs):
        if not self.fused:#1*1 conv
            rbr_1x1_output = self.rbr_1x1(inputs)
        else:
            rbr_1x1_output = None

        if self.rbr_dense is None:#3*3 conv
            dense_output = 0
        else:
            dense_output = self.rbr_dense(inputs)

        return rbr_1x1_output, dense_output

    def forward(self, inputs):
        if self.rbr_identity is None:
            id_out = 0
        else:
            id_out = self.rbr_identity(inputs)#BN

        rbr_1x1_output, drop_path_output = self._forward(inputs)

        if self.se_block:
            if self.rbr_identity is not None:
                id_out = id_out * self.se(id_out)###########

        if not self.fused:
            out = drop_path_output + rbr_1x1_output + id_out
        else:
            out = drop_path_output + id_out

        if self.se_block and (self.rbr_identity is None):
            out = out * self.se(inputs)############

        out = self.nonlinearity(out)

        if self.channel_shuffle:
            out = channel_shuffle(out, self.groups)

        return out

    def fuse_conv_bn(self, conv, bn):
        """
        # n,c,h,w - conv
        # n - bn (scale, bias, mean, var)

        if type(bn) is nn.Identity or type(bn) is None:
            return

        conv.weight
        running_mean = bn.running_mean
        running_var = bn.running_var
        gamma = bn.weight
        beta = bn.bias
        eps = bn.eps
        """
        std = (bn.running_var + bn.eps).sqrt()
        bias = bn.bias - bn.running_mean * bn.weight / std

        t = (bn.weight / std).reshape(-1, 1, 1, 1)
        weights = conv.weight * t

        bn = nn.Identity()
        conv = nn.Conv2d(in_channels=conv.in_channels,
                         out_channels=conv.out_channels,
                         kernel_size=conv.kernel_size,
                         stride=conv.stride,
                         padding=conv.padding,
                         dilation=conv.dilation,
                         groups=conv.groups,
                         bias=True,
                         padding_mode=conv.padding_mode)

        conv.weight = torch.nn.Parameter(weights)
        conv.bias = torch.nn.Parameter(bias)

        return conv

    def fuse_repvgg_block(self):
        self.rbr_dense = self.fuse_conv_bn(self.rbr_dense.conv, self.rbr_dense.bn)

        if isinstance(self.rbr_1x1, nn.Sequential) and isinstance(self.rbr_1x1[0], nn.AvgPool2d):
            self.rbr_1x1[1] = self.fuse_conv_bn(self.rbr_1x1[1].conv, self.rbr_1x1[1].bn)
            rbr_1x1_bias = self.rbr_1x1[1].bias

            weight_1x1_expanded = torch.nn.functional.interpolate(self.rbr_1x1[1].weight, scale_factor=2.0, mode='nearest')
            weight_1x1_expanded = weight_1x1_expanded / 4
            weight_1x1_expanded = torch.nn.functional.pad(weight_1x1_expanded, [1, 0, 1, 0])
        else:
            self.rbr_1x1 = self.fuse_conv_bn(self.rbr_1x1.conv, self.rbr_1x1.bn)
            rbr_1x1_bias = self.rbr_1x1.bias

            weight_1x1_expanded = torch.nn.functional.pad(self.rbr_1x1.weight, [1, 1, 1, 1])

        self.rbr_dense.weight = torch.nn.Parameter(self.rbr_dense.weight + weight_1x1_expanded)
        self.rbr_dense.bias = torch.nn.Parameter(self.rbr_dense.bias + rbr_1x1_bias)

        self.rbr_1x1 = nn.Identity()

        self.fused = True
